In [3]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt

import copy
import random

In [4]:
class cube_visualizer:
    #hardcoded he colors of the rubik cube
    color=[
        "#f0f0f0", #white
        "#00ff00", #green
        "#ff0000", #red
        "#0000ff", #blue
        "#ffa500", #orange
        "#ffff00" #yellow
    ]
    
    def add_surface(ax,x,y,z,direction,rgba): #helper function to add flat surface (with color) on a mathplotlib graph
        #(x,y,z) coordinate of origin of the surface
        #direction:
        #    0 - xy
        #    1 - yz
        #    2 - xz

        p1=np.array([[0,0],[1,1]])
        p2=np.array([[0,1],[0,1]])
        d=np.array([[0,0],[0,0]])

        if (direction==0):
            ax.plot_surface(p1+x,p2+y,d+z,color=rgba,shade=False)
        elif (direction==1):
            ax.plot_surface(d+x,p1+y,p2+z,color=rgba,shade=False)
        else :
            ax.plot_surface(p1+x,d+y,p2+z,color=rgba,shade=False)
            
    def plot_cube(ax,colors):
        #U
        cube_visualizer.add_surface(ax,0,1,2,0,cube_visualizer.color[colors[0][0]])
        cube_visualizer.add_surface(ax,1,1,2,0,cube_visualizer.color[colors[0][1]])
        cube_visualizer.add_surface(ax,1,0,2,0,cube_visualizer.color[colors[0][2]])
        cube_visualizer.add_surface(ax,0,0,2,0,cube_visualizer.color[colors[0][3]])
        
        #L
        cube_visualizer.add_surface(ax,0,1,1,1,cube_visualizer.color[colors[1][0]])
        cube_visualizer.add_surface(ax,0,0,1,1,cube_visualizer.color[colors[1][1]])
        cube_visualizer.add_surface(ax,0,0,0,1,cube_visualizer.color[colors[1][2]])
        cube_visualizer.add_surface(ax,0,1,0,1,cube_visualizer.color[colors[1][3]])
        
        #F
        cube_visualizer.add_surface(ax,0,0,1,2,cube_visualizer.color[colors[2][0]])
        cube_visualizer.add_surface(ax,1,0,1,2,cube_visualizer.color[colors[2][1]])
        cube_visualizer.add_surface(ax,1,0,0,2,cube_visualizer.color[colors[2][2]])
        cube_visualizer.add_surface(ax,0,0,0,2,cube_visualizer.color[colors[2][3]])
        
        #R
        cube_visualizer.add_surface(ax,2,0,1,1,cube_visualizer.color[colors[3][0]])
        cube_visualizer.add_surface(ax,2,1,1,1,cube_visualizer.color[colors[3][1]])
        cube_visualizer.add_surface(ax,2,1,0,1,cube_visualizer.color[colors[3][2]])
        cube_visualizer.add_surface(ax,2,0,0,1,cube_visualizer.color[colors[3][3]])
        
        #B
        cube_visualizer.add_surface(ax,1,2,1,2,cube_visualizer.color[colors[4][0]])
        cube_visualizer.add_surface(ax,0,2,1,2,cube_visualizer.color[colors[4][1]])
        cube_visualizer.add_surface(ax,0,2,0,2,cube_visualizer.color[colors[4][2]])
        cube_visualizer.add_surface(ax,1,2,0,2,cube_visualizer.color[colors[4][3]])
        
        #D
        cube_visualizer.add_surface(ax,0,0,0,0,cube_visualizer.color[colors[5][0]])
        cube_visualizer.add_surface(ax,1,0,0,0,cube_visualizer.color[colors[5][1]])
        cube_visualizer.add_surface(ax,1,1,0,0,cube_visualizer.color[colors[5][2]])
        cube_visualizer.add_surface(ax,0,1,0,0,cube_visualizer.color[colors[5][3]])
            
    def default_cube(): #just to test the color scheme on a 1x1x1 cube
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        cube_visualizer.add_surface(ax,0,0,1,0,cube_visualizer.color[0])
        cube_visualizer.add_surface(ax,0,0,0,1,cube_visualizer.color[1])
        cube_visualizer.add_surface(ax,0,0,0,2,cube_visualizer.color[2])
        cube_visualizer.add_surface(ax,1,0,0,1,cube_visualizer.color[3])
        cube_visualizer.add_surface(ax,0,1,0,2,cube_visualizer.color[4])
        cube_visualizer.add_surface(ax,0,0,0,0,cube_visualizer.color[5])

        plt.axis("off")

        plt.show()

To ensure that we are not double counting states that are identical up to rotation, we will only use ULF moves and their inverse. This will keep one of the corners of the cube in the same position and therefore each state that we traverse is unique.

In [5]:
class cube: #the class to represent the state of a cube
    #each sticker of the cube will be represented as a number and stored in a 6x4 array
    
    #faces
    #   0          U
    # 1 2 3 4    L F R B
    #   5          D
    
    #stickers
    #      0 1
    #      3 2
    #  4 5 8 9 12 13 16 17
    #  7 6 11 10 15 14 19 18
    #      20 21
    #      23 22
    
    #do dot product with hashing later
    hashp=6**np.arange(24,dtype=int)
    
    def __init__ (self,obj=None): # create
        if obj is None: #default cube
            self.arr=np.arange(24,dtype=int).reshape((6,4))
        else: #deep copy another cube object
            self.arr=np.copy(obj.arr)
    
    def get_colors(self): #the color is just the sticker id divided by 4
        return self.arr//4
    
    #helper function to rotate stickers
    def rotate(self,tup):
        for i in range(len(tup)-2,-1,-1):
            self.arr[tup[i][0]][tup[i][1]],self.arr[tup[i+1][0]][tup[i+1][1]]=self.arr[tup[i+1][0]][tup[i+1][1]],self.arr[tup[i][0]][tup[i][1]]
    
    #define the allowed moves (using quarter turn metric)
    def U(self):
        self.rotate([(0,0),(0,1),(0,2),(0,3)])
        self.rotate([(4,0),(3,0),(2,0),(1,0)])
        self.rotate([(4,1),(3,1),(2,1),(1,1)])
        
    def Up(self):
        self.U()
        self.U()
        self.U()
        
    def L(self):
        self.rotate([(1,0),(1,1),(1,2),(1,3)])
        self.rotate([(0,0),(2,0),(5,0),(4,2)])
        self.rotate([(0,3),(2,3),(5,3),(4,1)])
        
    def Lp(self):
        self.L()
        self.L()
        self.L()
    
    def F(self):
        self.rotate([(2,0),(2,1),(2,2),(2,3)])
        self.rotate([(0,3),(3,0),(5,1),(1,2)])
        self.rotate([(0,2),(3,3),(5,0),(1,1)])
        
    def Fp(self):
        self.F()
        self.F()
        self.F()
        
    def R(self):
        self.rotate([(3,0),(3,1),(3,2),(3,3)])
        self.rotate([(0,2),(4,0),(5,2),(2,2)])
        self.rotate([(0,1),(4,3),(5,1),(2,1)])
    
    def Rp(self):
        self.R()
        self.R()
        self.R()
        
    def B(self):
        self.rotate([(4,0),(4,1),(4,2),(4,3)])
        self.rotate([(0,1),(1,0),(5,3),(3,2)])
        self.rotate([(0,0),(1,3),(5,2),(3,1)])
    
    def Bp(self):
        self.B()
        self.B()
        self.B()
        
    def D(self):
        self.rotate([(5,0),(5,1),(5,2),(5,3)])
        self.rotate([(1,2),(2,2),(3,2),(4,2)])
        self.rotate([(1,3),(2,3),(3,3),(4,3)])
    
    def Dp(self):
        self.D()
        self.D()
        self.D()
        
    def move(self,face):
        if (face==0): self.U()
        if (face==1): self.Up()
        if (face==2): self.L()
        if (face==3): self.Lp()
        if (face==4): self.F()
        if (face==5): self.Fp()
        
    def scramble(self):
        for i in range(100):
            self.move(random.randrange(6))
            
        
    def plot(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        cube_visualizer.plot_cube(ax,self.get_colors())
        
        plt.axis("off")
        
        plt.show()
        
    def get_id(self):
        col=self.get_colors()
        return col.ravel().dot(cube.hashp)